<a href="https://colab.research.google.com/github/arshmehar/ssh_Indian_Ocean/blob/main/finssh_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Timeseries training using LSTMs

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import xarray as xr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, Callback


In [ ]:
path = "/content/drive/MyDrive/prerna_data/hadgem_aviso.nc"
ds   = xr.open_mfdataset(path, decode_times=False, engine="h5netcdf")

In [ ]:
ds   = ds.mean(dim="ensemble_member")

X_in = ds.DSL.fillna(0).values        # (T, 240, 360)#hadgem dsl array
y_in = ds.avisodsl.fillna(0).values

In [ ]:
T, H, W = X_in.shape
print(f"Full cube : T={T}, H={H}, W={W}")#t is time steops, lat, long

Full cube : T=264, H=240, W=360


In [ ]:
X_in = X_in.reshape(T, -1)
y_in = y_in.reshape(T, -1)

In [ ]:
X_grid = X_in.T     # shape: (H*W, T)
y_grid = y_in.T     # shape: (H*W, T)

In [ ]:
y_grid.shape

(86400, 264)

In [ ]:
n_pixels=X_grid.shape[0]
pix_ids      = np.arange(n_pixels)                 # 0 … 86 399
rng          = np.random.default_rng(42)        # reproducible
rng.shuffle(pix_ids) # permutes the list in‑place; every pixel gets a new random position.

In [ ]:
n_train      = int(0.80 * n_pixels)
n_val        = int(0.10 * n_pixels)

train_ids    = pix_ids[:n_train]
val_ids      = pix_ids[n_train:n_train+n_val]
test_ids     = pix_ids[n_train+n_val:]

In [ ]:
X_train = X_grid[train_ids]   # shape: (n_train, T)
y_train = y_grid[train_ids]

X_val = X_grid[val_ids]
y_val = y_grid[val_ids]

X_test = X_grid[test_ids]
y_test = y_grid[test_ids]

In [ ]:
print("Pixel‑only cubes  :")
for name, arr in [("train", X_train), ("val", X_val), ("test", X_test)]:
    print(f"{name:5s}  {arr.shape}")


Pixel‑only cubes  :
train  (69120, 264)
val    (8640, 264)
test   (8640, 264)


In [ ]:
def make_seq(data_grid, target_grid, steps):
    """
    Create time-windowed sequences from (n_gridpoints, time) data.

    data_grid:   (n, T)
    target_grid: (n, T)
    steps:       input window size

    Returns:
        X_seq: (n_samples, steps, 1)
        y_seq: (n_samples,)
    """
    Xs, ys = [], []

    for gridpoint in range(data_grid.shape[0]):
        for t in range(data_grid.shape[1] - steps):
            Xs.append(data_grid[gridpoint, t:t+steps])
            ys.append(target_grid[gridpoint, t+steps])  # predict the next timestep

    Xs = np.array(Xs)
    ys = np.array(ys)
    return Xs[..., np.newaxis], ys  # add a feature dim for LSTM


In [ ]:
timesteps = 10

X_tr, y_tr = make_seq(X_train, y_train, timesteps)
X_vl, y_vl = make_seq(X_val,   y_val,   timesteps)
X_te, y_te = make_seq(X_test,  y_test,  timesteps)

In [ ]:
print("train :", X_tr.shape, y_tr.shape)
print("val   :", X_vl.shape, y_vl.shape)
print("test  :", X_te.shape, y_te.shape)

Final shapes → LSTM:
train : (17556480, 10, 1) (17556480,)
val   : (2194560, 10, 1) (2194560,)
test  : (2194560, 10, 1) (2194560,)


In [ ]:
#training

In [ ]:


# Define and compile model
model = Sequential([
    LSTM(64, input_shape=(10, 1)),
    Dense(1)
])

model.compile(optimizer=Adam(1e-4), loss='mse')


In [ ]:

# Directory to save best models
save_dir = '/home/cccr_rnd/arshmehar/models'
os.makedirs(save_dir, exist_ok=True)

# Modify the checkpoint callback to include formatted filename
checkpoint_cb = ModelCheckpoint(
    filepath=os.path.join(save_dir, "best_model_epoch{epoch:04d}_val{val_loss:.6f}.h5"),  # Keep your existing format
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# Custom callback to print a message when training completes
class TrainingCompleteCallback(Callback):
    def on_train_end(self, logs=None):
        print("✅ Training completed successfully!")



In [ ]:
history = model.fit(
    X_tr, y_tr,
    validation_data=(X_vl, y_vl),
    epochs=2,
    batch_size=2048,  # You can tune this based on memory
    callbacks=[checkpoint_cb, TrainingCompleteCallback()]
)


Epoch 1/2
8573/8573 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.0068
Epoch 1: val_loss improved from 0.00696 to 0.00674, saving model to /home/cccr_rnd/arshmehar/models/best_model_epoch001_val0.006738.h5


8573/8573 ━━━━━━━━━━━━━━━━━━━━ 1083s 126ms/step - loss: 0.0068 - val_loss: 0.0067
Epoch 2/2
8572/8573 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 0.0066
Epoch 2: val_loss improved from 0.00674 to 0.00662, saving model to /home/cccr_rnd/arshmehar/models/best_model_epoch002_val0.006616.h5


8573/8573 ━━━━━━━━━━━━━━━━━━━━ 1063s 124ms/step - loss: 0.0066 - val_loss: 0.0066
✅ Training completed successfully!


In [ ]:
loss = model.evaluate(X_test, y_test, batch_size=2048)
print(f"Loss: {loss}")


1072/1072 ━━━━━━━━━━━━━━━━━━━━ 67s 63ms/step - loss: 0.0066
Loss: 0.006532410625368357
